In [72]:
import numpy as np
import torch
import numba as nb
import torchquad
from scipy import optimize, integrate, linalg
from classes import Region, Coordinate, Demands_generator, Demand

# Scipy Implementation for Problem 7

tol = 1e-4
# Instrumental functions for scipy integration
@nb.jit(nopython=True)
def norm_func(x, y):
    return np.sqrt(np.sum(np.square(x - y)))

@nb.jit(nopython=True)
def modified_norm(x_cdnt, i, demands_locations, lambdas):
    return norm_func(x_cdnt, demands_locations[i]) - lambdas[i]

@nb.jit(nopython=True)
def region_indicator(i, x_cdnt, lambdas, demands_locations):
    i_modified_norm = modified_norm(x_cdnt, i, demands_locations, lambdas)
    for j in range(len(lambdas)):
        if i_modified_norm > modified_norm(x_cdnt, j, demands_locations, lambdas):
            return 0
    return 1

@nb.jit(nopython=True)
def categorize_x(x_cdnt, demands_locations, lambdas, v):
    modified_norms = np.array([modified_norm(x_cdnt, i, demands_locations, lambdas) for i in range(demands_locations.shape[0])])
    i = np.argmin(modified_norms)
    return demands_locations[i], v[i+1]

# Integrands of function and derivatives

@nb.njit
def integrand_scipy(r, theta, lambdas, v, demands_locations):
    x_cdnt = np.array([r*np.cos(theta), r*np.sin(theta)])
    xi, vi = categorize_x(x_cdnt, demands_locations, lambdas, v)
    raw_intgrd = 1 / (v[0]*norm_func(x_cdnt, xi) + vi)
    return r*raw_intgrd

@nb.njit
def jac_integrand0_scipy(r, theta, lambdas, v, demands_locations):
    x_cdnt = np.array([r*np.cos(theta), r*np.sin(theta)])
    xi, vi = categorize_x(x_cdnt, demands_locations, lambdas, v)
    the_norm = norm_func(x_cdnt, xi)
    return -r * the_norm/pow(v[0]*the_norm + vi, 2)

'''NOTE: i is the index of v, whose values range from 0 to n
      But j is the index of regions, whose values range from 1 to n.
      For indexes of demands_locations (0 to n-1), we need to offset j by subtracting one.'''

@nb.njit
def jac_integrandj_scipy(r, theta, lambdas, v, demands_locations, j):
    x_cdnt = np.array([r*np.cos(theta), r*np.sin(theta)])
    if region_indicator(j-1, x_cdnt, lambdas, demands_locations) == 0: return 0
    return -r / pow(v[0] * norm_func(x_cdnt, demands_locations[j-1]) + v[j], 2)


def objective_function_scipy(v, demands_locations, lambdas, t, region_radius, thetarange):
    print(f"DEBUG: v is {v, type(v[1])}.")
    start, end = thetarange
    sum_integral, error = integrate.dblquad(integrand_scipy, start, end, lambda _: 0, lambda _: region_radius, args=(lambdas, v, demands_locations), epsabs=tol)
    return 1/4*sum_integral + v[0]*t + np.mean(v[1:])


def objective_jac_scipy(v: list[float], demands_locations: list[list[float]], lambdas: list[float], t: float, region_radius, thetarange):
    start, end = thetarange
    n = demands_locations.shape[0]
    jac = np.zeros(n + 1)
    jac[0] = 1/4 * integrate.dblquad(jac_integrand0_scipy, start, end, lambda _: 0, lambda _: region_radius, args=(lambdas, v, demands_locations), epsabs=tol)[0] + t
    for j in range(1, n+1):
        jac[j] = 1/4 * integrate.dblquad(jac_integrandj_scipy, start, end, lambda _: 0, lambda _: region_radius, args=(lambdas, v, demands_locations, j), epsabs=tol)[0] + 1/n
    print(f"DEBUG: jac is {jac}.")
    return jac


def minimize_problem7_scipy(lambdas, demands, thetarange, t, region_radius):
    # constraints = [optimize.NonlinearConstraint(lambda v: constraint_func(lambdas, demands, v, region_radius), 0, np.inf)]
    demands_locations = np.array([demands[i].get_cdnt() for i in range(len(demands))])
    bounds = optimize.Bounds(1e-12, np.inf)
    result = optimize.minimize(objective_function_scipy, x0=np.append(1e-6, np.ones(demands.shape[0])), args=(demands_locations, lambdas, t, region_radius, thetarange), jac=objective_jac_scipy, method='SLSQP',  bounds=bounds, options={'ftol': tol, 'disp': True})
    return result.x, result.fun

region = Region(1)
depot = Coordinate(2, 0.3)
t, epsilon = 0.25, 0.1

thetarange = (3.147444264116321 - 2*np.pi, 0.00012460881117814946)
demands = np.array([Demand(Coordinate(r = 0.1802696888767692, theta = 4.768809396779301), 1),
           Demand(Coordinate(r = 0.019475241487624584, theta = 5.1413757057591045), 1),
           Demand(Coordinate(r = 0.012780814590608647, theta = 4.478189127165961), 1),
           Demand(Coordinate(r = 0.4873716073198716, theta = 3.7670422583826495), 1),
           Demand(Coordinate(r = 0.10873607186897494, theta = 5.328009178184025), 1),
           Demand(Coordinate(r = 0.8939041702850812, theta = 4.5103794163992506), 1),
           Demand(Coordinate(r = 0.8571542470728296, theta = 3.7828800007876318), 1),
           Demand(Coordinate(r = 0.16508661759522714, theta = 3.4707299112070515), 1),
           Demand(Coordinate(r = 0.6323340138234961, theta = 5.963386240530588), 1),
           Demand(Coordinate(r = 0.020483612791232675, theta = 6.19945137229428), 1),
           Demand(Coordinate(r = 0.15791230667493694, theta = 5.004153427569559), 1)])
demands_locations = np.array([demands[i].get_cdnt() for i in range(len(demands))])
lambdas = np.zeros(demands.shape[0])

v_scipy, obj_scipy = minimize_problem7_scipy(lambdas, demands, thetarange, t, region.radius)
# objective_function(np.append(1e-6, np.ones(demands.shape[0])), demands_locations, lambdas, t, region.radius, thetarange), objective_function_scipy(np.append(1e-6, np.ones(demands.shape[0])), demands_locations, lambdas, t, region.radius, thetarange)

DEBUG: v is (array([1.e-06, 1.e+00, 1.e+00, 1.e+00, 1.e+00, 1.e+00, 1.e+00, 1.e+00,
       1.e+00, 1.e+00, 1.e+00, 1.e+00]), <class 'numpy.float64'>).


/Users/miaoyidi/miniconda3/envs/HCT/lib/python3.11/site-packages/scipy/integrate/_quadpack_py.py:1233: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
/Users/miaoyidi/miniconda3/envs/HCT/lib/python3.11/site-packages/scipy/integrate/_quadpack_py.py:1233: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


DEBUG: jac is [ 0.15237174  0.05542283  0.09009648  0.08939359  0.0334797   0.07921427
  0.00754794  0.02935291  0.07738297 -0.02124352  0.09025808  0.07757108].
DEBUG: v is (array([1.00003239e-12, 9.44577173e-01, 9.09903524e-01, 9.10606415e-01,
       9.66520303e-01, 9.20785726e-01, 9.92452058e-01, 9.70647090e-01,
       9.22617031e-01, 1.02124352e+00, 9.09741921e-01, 9.22428915e-01]), <class 'numpy.float64'>).
DEBUG: jac is [ 0.14951366  0.05113648  0.08992754  0.08908092  0.02943194  0.07711544
  0.00627502  0.02557372  0.07501866 -0.0166262   0.09012038  0.07523316].
DEBUG: v is (array([1.00000000e-12, 6.87506923e-01, 4.58009720e-01, 4.62963311e-01,
       8.18522226e-01, 5.33224947e-01, 9.60887936e-01, 8.42043447e-01,
       5.45585984e-01, 1.10490648e+00, 4.56879891e-01, 5.44320683e-01]), <class 'numpy.float64'>).
DEBUG: jac is [ 0.11997267  0.01583285  0.08703979  0.08383296  0.00519164  0.04977739
  0.00062244  0.00409269  0.04546713 -0.00095775  0.08778151  0.04581072].
DEBUG:

/Users/miaoyidi/miniconda3/envs/HCT/lib/python3.11/site-packages/scipy/integrate/_quadpack_py.py:1233: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,
/Users/miaoyidi/miniconda3/envs/HCT/lib/python3.11/site-packages/scipy/integrate/_quadpack_py.py:1233: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


DEBUG: v is (array([1.00000551e-12, 6.63001776e-01, 4.12208748e-01, 4.16666980e-01,
       8.05205597e-01, 4.91043154e-01, 9.58145159e-01, 8.30556118e-01,
       5.05148590e-01, 1.11203889e+00, 4.11191902e-01, 5.03724552e-01]), <class 'numpy.float64'>).
DEBUG: jac is [1.13988924e-01 1.01806725e-02 8.61360241e-02 8.21736853e-02
 2.33284677e-03 4.24076863e-02 1.04793274e-04 1.67384425e-03
 3.79012071e-02 2.16953115e-04 8.70479885e-02 3.82480396e-02].
DEBUG: v is (array([1.00006024e-12, 4.90220573e-01, 1.00003339e-12, 1.00000000e-12,
       7.12222424e-01, 1.53980672e-01, 9.38720583e-01, 7.50239664e-01,
       1.88348893e-01, 1.16356664e+00, 1.00000000e-12, 1.85251626e-01]), <class 'numpy.float64'>).


/Users/miaoyidi/miniconda3/envs/HCT/lib/python3.11/site-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


DEBUG: v is (array([1.00001098e-12, 6.45723656e-01, 3.70987873e-01, 3.75000282e-01,
       7.95907279e-01, 4.57336906e-01, 9.56202702e-01, 8.22524473e-01,
       4.73468620e-01, 1.11719167e+00, 3.70072712e-01, 4.71877259e-01]), <class 'numpy.float64'>).
DEBUG: jac is [ 1.08597415e-01  5.80292282e-03  8.50120953e-02  8.01253405e-02
  2.51065626e-04  3.49937594e-02 -2.64504093e-04 -7.73636691e-05
  3.05704029e-02  1.05300350e-03  8.61419897e-02  3.09008899e-02].
DEBUG: v is (array([1.00003831e-12, 5.25849481e-01, 1.00000000e-12, 1.00000000e-12,
       7.31637644e-01, 1.96752982e-01, 9.42542985e-01, 7.66896298e-01,
       2.33050359e-01, 1.15422046e+00, 1.00003339e-12, 2.29892168e-01]), <class 'numpy.float64'>).
DEBUG: v is (array([1.00001371e-12, 6.33736238e-01, 3.33889086e-01, 3.37500254e-01,
       7.89480316e-01, 4.31278514e-01, 9.54836730e-01, 8.16961655e-01,
       4.49426794e-01, 1.12089454e+00, 3.33065441e-01, 4.47678750e-01]), <class 'numpy.float64'>).
DEBUG: jac is [ 0.10392231 

In [14]:
# Torchquad Implementation for Problem 7
def integrand(X, lambdas, v, demands_locations):
    '''
    X is a n-by-2 matrix, the first column is r, the second column is theta.
    '''
    dtype = X.dtype
    print(f"lambdas is {lambdas}., v0 is {v[0]}, v1 is {v[1:]}.")
    lambdas, v1, v0 = torch.tensor(lambdas, dtype=dtype), torch.tensor(v[1:], dtype=dtype), v[0]
    # print(f"lambdas is {lambdas}., v0 is {v0}, v1 is {v1}.")
    demands_locations = torch.tensor(demands_locations, dtype=dtype)
    x_cdnt = X.clone()
    x_cdnt[:, 0] = X[:, 0]*torch.cos(x_cdnt[:, 1])
    x_cdnt[:, 1] = X[:, 0]*torch.sin(x_cdnt[:, 1])
    norms = torch.cdist(x_cdnt, demands_locations, p=2)
    modified_norms, modified_norms_indices = torch.min(norms - lambdas, dim=1)
    raw_intgrd = 1 / (v0*norms[torch.arange(norms.shape[0]), modified_norms_indices] + v1[modified_norms_indices])
    if raw_intgrd.is_complex(): 
        # print(f"raw_intgrd is complex. {raw_intgrd}")
        raise ValueError
    return X[:, 0]*raw_intgrd

def jac_integrand0(X, lambdas, v, demands_locations):
    '''
    X is a n-by-2 matrix, the first column is r, the second column is theta.
    '''
    dtype = X.dtype
    lambdas, v1, v0 = torch.tensor(lambdas, dtype=dtype), torch.tensor(v[1:], dtype=dtype), v[0]
    demands_locations = torch.tensor(demands_locations, dtype=dtype)
    x_cdnt = X.clone()
    x_cdnt[:, 0] = X[:, 0]*torch.cos(x_cdnt[:, 1])
    x_cdnt[:, 1] = X[:, 0]*torch.sin(x_cdnt[:, 1])
    norms = torch.cdist(x_cdnt, demands_locations, p=2)
    modified_norms, modified_norms_indices = torch.min(norms - lambdas, dim=1)
    cooresponding_norms = norms[torch.arange(norms.shape[0]), modified_norms_indices]
    raw_intgrd = cooresponding_norms / torch.square(v0*cooresponding_norms + v1[modified_norms_indices])
    return -X[:, 0] * raw_intgrd

def jac_integrandj(X, lambdas, v, demands_locations, j):
    '''
    X is a n-by-2 matrix, the first column is r, the second column is theta.
    '''
    dtype = X.dtype
    lambdas, v1, v0 = torch.tensor(lambdas, dtype=dtype), torch.tensor(v[1:], dtype=dtype), v[0]
    demands_locations = torch.tensor(demands_locations, dtype=dtype)
    x_cdnt = X.clone()
    x_cdnt[:, 0] = X[:, 0]*torch.cos(x_cdnt[:, 1])
    x_cdnt[:, 1] = X[:, 0]*torch.sin(x_cdnt[:, 1])
    norms = torch.cdist(x_cdnt, demands_locations, p=2)
    modified_norms, modified_norms_indices = torch.min(norms - lambdas, dim=1)
    maskj = modified_norms_indices != j-1
    cooresponding_norms = norms[torch.arange(norms.shape[0]), modified_norms_indices]
    raw_intgrd = 1 / torch.square(v0*cooresponding_norms + v1[modified_norms_indices])
    masked_intgrd = -X[:, 0] * raw_intgrd
    masked_intgrd[maskj] = 0
    return masked_intgrd

def objective_function(v, demands_locations, lambdas, t, region_radius, thetarange):
    # print(f"DEBUG: v is {v, type(v[1])}.")
    # v = np.real(v)
    start, end = thetarange
    simpson = torchquad.Simpson()
    sum_integral = simpson.integrate(lambda X: integrand(X, lambdas, v, demands_locations), dim=2, N=10000001, integration_domain=[[0, region_radius], [start, end]], backend='torch').item()
    # print(f"sum_integral is {sum_integral}. with type {type(sum_integral)}")
    return 1/4*sum_integral + v[0]*t + np.mean(v[1:])

def objective_jac(v, demands_locations, lambdas, t, region_radius, thetarange):
    start, end = thetarange
    n = demands_locations.shape[0]
    jac = np.zeros(n + 1)
    simpson = torchquad.Simpson()
    jac[0] = 1/4 * simpson.integrate(lambda X: jac_integrand0(X, lambdas, v, demands_locations), dim=2, N=10000001, integration_domain=[[0, region_radius], [start, end]], backend='torch').item() + t

    # The precompiled version is even slower, so we don't use it.
    # for j in torch.range(1, n): # notice torch.range is inclusive on both ends, hence we use n as upper bound
    #     if j == 1:
    #         simpsonj_compiled = torch.jit.trace(
    #             lambda vj_index: simpson.integrate(lambda X: jac_integrandj(X, lambdas, v, demands_locations, vj_index), dim=2, N=10000001, integration_domain=[[0, region_radius], [start, end]], backend='torch'),
    #             (j,)
    #                 )
    #     jac[int(j.item())] = 1/4 * simpsonj_compiled(j).item() + 1/n


    for j in range(1, n+1):
        jac[j] = 1/4 * simpson.integrate(lambda X: jac_integrandj(X, lambdas, v, demands_locations, j), dim=2, N=10000001, integration_domain=[[0, region_radius], [start, end]], backend='torch').item() + 1/n
    print(f"DEBUG: jac is {jac}.")
    return jac


def minimize_problem7(lambdas, demands, thetarange, t, region_radius, tol=1e-4):
    # constraints = [optimize.NonlinearConstraint(lambda v: constraint_func(lambdas, demands, v, region_radius), 0, np.inf)]
    demands_locations = np.array([demands[i].get_cdnt() for i in range(len(demands))])
    bounds = optimize.Bounds(0, np.inf)
    result = optimize.minimize(objective_function, x0=np.append(1e-6, np.ones(demands.shape[0])), args=(demands_locations, lambdas, t, region_radius, thetarange), jac=objective_jac, method='SLSQP',  bounds=bounds, options={'ftol': tol, 'disp': True})
    return result.x, result.fun


region = Region(1)
depot = Coordinate(2, 0.3)
t, epsilon = 0.25, 0.1

thetarange = (3.147444264116321 - 2*np.pi, 0.00012460881117814946)
demands = np.array([Demand(Coordinate(r = 0.1802696888767692, theta = 4.768809396779301), 1),
           Demand(Coordinate(r = 0.019475241487624584, theta = 5.1413757057591045), 1),
           Demand(Coordinate(r = 0.012780814590608647, theta = 4.478189127165961), 1),
           Demand(Coordinate(r = 0.4873716073198716, theta = 3.7670422583826495), 1),
           Demand(Coordinate(r = 0.10873607186897494, theta = 5.328009178184025), 1),
           Demand(Coordinate(r = 0.8939041702850812, theta = 4.5103794163992506), 1),
           Demand(Coordinate(r = 0.8571542470728296, theta = 3.7828800007876318), 1),
           Demand(Coordinate(r = 0.16508661759522714, theta = 3.4707299112070515), 1),
           Demand(Coordinate(r = 0.6323340138234961, theta = 5.963386240530588), 1),
           Demand(Coordinate(r = 0.020483612791232675, theta = 6.19945137229428), 1),
           Demand(Coordinate(r = 0.15791230667493694, theta = 5.004153427569559), 1)])
demands_locations = np.array([demands[i].get_cdnt() for i in range(len(demands))])
lambdas = np.zeros(demands.shape[0])

v_torch, obj_torch = minimize_problem7(lambdas, demands, thetarange, t, region.radius)

/Users/miaoyidi/miniconda3/envs/HCT/lib/python3.11/site-packages/torchquad/integration/simpson.py:77: UserWarning: N per dimension cannot be even due to necessary subdivisions. N per dim will now be changed to the next lower integer, i.e. 3162 -> 3161.
  warnings.warn(


lambdas is [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]., v0 is 1e-06, v1 is [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.].


/Users/miaoyidi/miniconda3/envs/HCT/lib/python3.11/site-packages/torchquad/integration/utils.py:255: UserWarning: DEPRECATION WARNING: In future versions of torchquad, an array-like object will be returned.
  warnings.warn(


DEBUG: jac is [ 0.15237721  0.05538272  0.09007328  0.08939176  0.03337792  0.0790582
  0.00755116  0.0293338   0.07736579 -0.02126194  0.09025265  0.07751708].
lambdas is [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]., v0 is 0.0, v1 is [0.94461728 0.90992672 0.91060824 0.96662208 0.9209418  0.99244884
 0.9706662  0.92263421 1.02126194 0.90974735 0.92248292].
DEBUG: jac is [ 0.14952369  0.05109478  0.08989961  0.08907924  0.02933614  0.07693618
  0.00627782  0.0255559   0.07499926 -0.01663999  0.09011595  0.07517182].
lambdas is [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]., v0 is 0.0, v1 is [0.68775439 0.45816959 0.46297006 0.81910424 0.53427807 0.96087036
 0.84215101 0.54569756 1.10499513 0.45690403 0.54467965].
DEBUG: jac is [ 0.12006699  0.01580141  0.08692749  0.08383007  0.0051609   0.049393
  0.00062368  0.0040877   0.04542907 -0.00095807  0.08776465  0.04576885].
lambdas is [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]., v0 is 1.3768674703441498e-16, v1 is [4.42965117e-01 0.00000000e+00 1.11022302e-16 6.86412

In [74]:
print(f"v_scipy is {v_scipy}, obj_scipy is {obj_scipy}.")
print(f"v_torch is {v_torch}, obj_torch is {obj_torch}.")

v_scipy is [1.00001432e-12 6.14831292e-01 9.65512921e-02 1.34037756e-01
 7.83611262e-01 3.58982717e-01 9.53970984e-01 8.12295322e-01
 3.85090314e-01 1.12336499e+00 8.45504191e-02 3.82805938e-01], obj_scipy is 1.0460955907232967.
v_torch is [0.         0.61838486 0.09597219 0.12811131 0.78715139 0.36121432
 0.95469298 0.815      0.38568693 1.12128326 0.09069173 0.38356752], obj_torch is 1.0460491390275553.


In [75]:
v = np.array([3.46391326e-18, 6.14822318e-01, 9.65945463e-02, 1.34070553e-01,
        7.83603013e-01, 3.58985231e-01, 9.53966571e-01, 8.12286499e-01,
        3.85096938e-01, 1.12337054e+00, 8.45804876e-02, 3.82777536e-01])
objective_function(v, demands_locations, lambdas, t, region.radius, thetarange), objective_function_scipy(v, demands_locations, lambdas, t, region.radius, thetarange)

lambdas is [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]., v0 is 3.46391326e-18, v1 is [0.61482232 0.09659455 0.13407055 0.78360301 0.35898523 0.95396657
 0.8122865  0.38509694 1.12337054 0.08458049 0.38277754].
DEBUG: v is (array([3.46391326e-18, 6.14822318e-01, 9.65945463e-02, 1.34070553e-01,
       7.83603013e-01, 3.58985231e-01, 9.53966571e-01, 8.12286499e-01,
       3.85096938e-01, 1.12337054e+00, 8.45804876e-02, 3.82777536e-01]), <class 'numpy.float64'>).


(1.046061192201905, 1.0460958165639682)

In [4]:
# Scipy implementation for Problem 14
import numpy as np
import torch
import numba as nb
import torchquad
from scipy import optimize, integrate, linalg
from classes import Region, Coordinate, Demands_generator, Demand

tol = 1e-3
@nb.jit(nopython=True)
def norm_func(x, y):
    return np.sqrt(np.sum(np.square(x - y)))

@nb.jit(nopython=True)
def min_modified_norm(x_cdnt, demands_locations, lambdas):
    n = demands_locations.shape[0]
    norms = np.array([norm_func(x_cdnt, demands_locations[i]) - lambdas[i] for i in range(n)])
    return np.min(norms)

@nb.njit
def integrand_scipy(r: float, theta: float, v, demands_locations, lambdas):
    # Calculate a list of ||x-xi|| - lambda_i
    x_cdnt = np.array([r*np.cos(theta), r*np.sin(theta)])
    raw_intgrd = 1/(4*(v[0]*min_modified_norm(x_cdnt, demands_locations, lambdas) + v[1]))
    return raw_intgrd*r    # r as Jacobian

@nb.njit
def jac_integrand0_scipy(r: float, theta: float, v, demands_locations, lambdas):
    x_cdnt = np.array([r*np.cos(theta), r*np.sin(theta)])
    the_min_modified_norm = min_modified_norm(x_cdnt, demands_locations, lambdas)
    raw_intgrd = -4*the_min_modified_norm / pow(4*(v[0]*the_min_modified_norm + v[1]), 2)
    return raw_intgrd*r

@nb.njit
def jac_integrand1_scipy(r: float, theta: float, v, demands_locations, lambdas):
    x_cdnt = np.array([r*np.cos(theta), r*np.sin(theta)])
    the_min_modified_norm = min_modified_norm(x_cdnt, demands_locations, lambdas)
    raw_intgrd = -4 / pow(4*(v[0]*the_min_modified_norm + v[1]), 2)
    return raw_intgrd*r

def objective_function_scipy(v, demands_locations, lambdas, t, region_radius, thetarange):
    start, end = thetarange
    area, error = integrate.dblquad(integrand_scipy, start, end, lambda _: 0, lambda _: region_radius, args=(v, demands_locations, lambdas), epsabs=tol)
    return area + v[0]*t + v[1]

def objective_jac_scipy(v, demands_locations, lambdas, t, region_radius, thetarange):
    start, end = thetarange
    area0, error0 = integrate.dblquad(jac_integrand0_scipy, start, end, lambda _: 0, lambda _: region_radius, args=(v, demands_locations, lambdas), epsabs=tol)
    area1, error1 = integrate.dblquad(jac_integrand1_scipy, start, end, lambda _: 0, lambda _: region_radius, args=(v, demands_locations, lambdas), epsabs=tol)
    return np.array([area0 + t, area1 + 1])

def constraint_and_jac(demands_locations, lambdas, region_radius):
    return np.array([min(-lambdas), 1]), np.array([min(-lambdas), 1])

def minimize_problem14_scipy(demands, thetarange, lambdas, t, region_radius):
    demands_locations = np.array([demands[i].get_cdnt() for i in range(len(demands))])
    constraint_coeff, constraint_jac = constraint_and_jac(demands_locations, lambdas, region_radius)
    constraints_dict = {'type': 'ineq', 'fun': lambda v: constraint_coeff @ v, 'jac': lambda _: constraint_jac}
    bound = optimize.Bounds(0.0001, np.inf)
    result = optimize.minimize(objective_function_scipy, x0=np.array([0.0001, 1]), args=(demands_locations, lambdas, t, region_radius, thetarange), jac=objective_jac_scipy, method='SLSQP', bounds=bound, constraints=constraints_dict)
    return result.x, result.fun

region = Region(1)
depot = Coordinate(2, 0.3)
t, epsilon = 0.25, 0.1

thetarange = (3.147444264116321 - 2*np.pi, 0.00012460881117814946)
demands = np.array([Demand(Coordinate(r = 0.1802696888767692, theta = 4.768809396779301), 1),
           Demand(Coordinate(r = 0.019475241487624584, theta = 5.1413757057591045), 1),
           Demand(Coordinate(r = 0.012780814590608647, theta = 4.478189127165961), 1),
           Demand(Coordinate(r = 0.4873716073198716, theta = 3.7670422583826495), 1),
           Demand(Coordinate(r = 0.10873607186897494, theta = 5.328009178184025), 1),
           Demand(Coordinate(r = 0.8939041702850812, theta = 4.5103794163992506), 1),
           Demand(Coordinate(r = 0.8571542470728296, theta = 3.7828800007876318), 1),
           Demand(Coordinate(r = 0.16508661759522714, theta = 3.4707299112070515), 1),
           Demand(Coordinate(r = 0.6323340138234961, theta = 5.963386240530588), 1),
           Demand(Coordinate(r = 0.020483612791232675, theta = 6.19945137229428), 1),
           Demand(Coordinate(r = 0.15791230667493694, theta = 5.004153427569559), 1)])
demands_locations = np.array([demands[i].get_cdnt() for i in range(len(demands))])
lambdas = np.zeros(demands.shape[0])

v_scipy, obj_scipy = minimize_problem14_scipy(demands, thetarange, lambdas, t, region.radius)

In [15]:
# Torchquad implementation for Problem 14
tol = 1e-3
def integrand(X, v, demands_locations, lambdas):
    '''
    X is a n-by-2 matrix, the first column is r, the second column is theta.
    '''
    dtype = X.dtype
    lambdas, v1, v0 = torch.tensor(lambdas, dtype=dtype), torch.tensor(v[1:], dtype=dtype), v[0]
    demands_locations = torch.tensor(demands_locations, dtype=dtype)
    x_cdnt = X.clone()
    x_cdnt[:, 0] = X[:, 0]*torch.cos(x_cdnt[:, 1])
    x_cdnt[:, 1] = X[:, 0]*torch.sin(x_cdnt[:, 1])
    norms = torch.cdist(x_cdnt, demands_locations, p=2)
    modified_norms, modified_norms_indices = torch.min(norms - lambdas, dim=1)
    raw_intgrd = 1 / (v[0]*modified_norms + v[1])
    return X[:, 0]*raw_intgrd    # r as Jacobian


def jac_integrand0(X, v, demands_locations, lambdas):
    '''
    X is a n-by-2 matrix, the first column is r, the second column is theta.
    '''
    dtype = X.dtype
    lambdas, v1, v0 = torch.tensor(lambdas, dtype=dtype), torch.tensor(v[1:], dtype=dtype), v[0]
    demands_locations = torch.tensor(demands_locations, dtype=dtype)
    x_cdnt = X.clone()
    x_cdnt[:, 0] = X[:, 0]*torch.cos(x_cdnt[:, 1])
    x_cdnt[:, 1] = X[:, 0]*torch.sin(x_cdnt[:, 1])
    norms = torch.cdist(x_cdnt, demands_locations, p=2)
    modified_norms, modified_norms_indices = torch.min(norms - lambdas, dim=1)
    raw_intgrd = -modified_norms / torch.square(v[0]*modified_norms + v[1])
    return X[:, 0]*raw_intgrd


def jac_integrand1(X, v, demands_locations, lambdas):
    '''
    X is a n-by-2 matrix, the first column is r, the second column is theta.
    '''
    dtype = X.dtype
    lambdas, v1, v0 = torch.tensor(lambdas, dtype=dtype), torch.tensor(v[1:], dtype=dtype), v[0]
    demands_locations = torch.tensor(demands_locations, dtype=dtype)
    x_cdnt = X.clone()
    x_cdnt[:, 0] = X[:, 0]*torch.cos(x_cdnt[:, 1])
    x_cdnt[:, 1] = X[:, 0]*torch.sin(x_cdnt[:, 1])
    norms = torch.cdist(x_cdnt, demands_locations, p=2)
    modified_norms, modified_norms_indices = torch.min(norms - lambdas, dim=1)
    raw_intgrd = -1 / torch.square(v[0]*modified_norms + v[1])
    return X[:, 0]*raw_intgrd

def objective_function(v, demands_locations, lambdas, t, region_radius, thetarange):
    start, end = thetarange
    simpson = torchquad.Simpson()
    area = simpson.integrate(lambda X: integrand(X, v, demands_locations, lambdas), dim=2, N=1000001, integration_domain=[[0, region_radius], [start, end]], backend='torch').item()
    return area/4 + v[0]*t + v[1]

def objective_jac(v, demands_locations, lambdas, t, region_radius, thetarange):
    start, end = thetarange
    simpson = torchquad.Simpson()
    area0 = simpson.integrate(lambda X: jac_integrand0(X, v, demands_locations, lambdas), dim=2, N=1000001, integration_domain=[[0, region_radius], [start, end]], backend='torch').item()
    area1 = simpson.integrate(lambda X: jac_integrand1(X, v, demands_locations, lambdas), dim=2, N=1000001, integration_domain=[[0, region_radius], [start, end]], backend='torch').item()
    return np.array([area0/4 + t, area1/4 + 1])

def constraint_and_jac(demands_locations, lambdas, region_radius):
    return np.array([min(-lambdas), 1]), np.array([min(-lambdas), 1])

def minimize_problem14(demands, thetarange, lambdas, t, region_radius):
    demands_locations = np.array([demands[i].get_cdnt() for i in range(len(demands))])
    constraint_coeff, constraint_jac = constraint_and_jac(demands_locations, lambdas, region_radius)
    constraints_dict = {'type': 'ineq', 'fun': lambda v: constraint_coeff @ v, 'jac': lambda _: constraint_jac}
    bound = optimize.Bounds(0, np.inf)
    result = optimize.minimize(objective_function, x0=np.array([0., 1]), args=(demands_locations, lambdas, t, region_radius, thetarange), jac=objective_jac, method='SLSQP', bounds=bound, constraints=constraints_dict)
    return result.x, result.fun

region = Region(1)
depot = Coordinate(2, 0.3)
t, epsilon = 0.25, 0.1

thetarange = (3.147444264116321 - 2*np.pi, 0.00012460881117814946)
demands = np.array([Demand(Coordinate(r = 0.1802696888767692, theta = 4.768809396779301), 1),
           Demand(Coordinate(r = 0.019475241487624584, theta = 5.1413757057591045), 1),
           Demand(Coordinate(r = 0.012780814590608647, theta = 4.478189127165961), 1),
           Demand(Coordinate(r = 0.4873716073198716, theta = 3.7670422583826495), 1),
           Demand(Coordinate(r = 0.10873607186897494, theta = 5.328009178184025), 1),
           Demand(Coordinate(r = 0.8939041702850812, theta = 4.5103794163992506), 1),
           Demand(Coordinate(r = 0.8571542470728296, theta = 3.7828800007876318), 1),
           Demand(Coordinate(r = 0.16508661759522714, theta = 3.4707299112070515), 1),
           Demand(Coordinate(r = 0.6323340138234961, theta = 5.963386240530588), 1),
           Demand(Coordinate(r = 0.020483612791232675, theta = 6.19945137229428), 1),
           Demand(Coordinate(r = 0.15791230667493694, theta = 5.004153427569559), 1)])
demands_locations = np.array([demands[i].get_cdnt() for i in range(len(demands))])
lambdas = np.zeros(demands.shape[0])

v_torch, obj_torch = minimize_problem14(demands, thetarange, lambdas, t, region.radius)

/Users/miaoyidi/miniconda3/envs/HCT/lib/python3.11/site-packages/torchquad/integration/simpson.py:77: UserWarning: N per dimension cannot be even due to necessary subdivisions. N per dim will now be changed to the next lower integer, i.e. 1000 -> 999.
  warnings.warn(


In [6]:
print(f"v_scipy is {v_scipy}, obj_scipy is {obj_scipy}.")
print(f"v_torch is {v_torch}, obj_torch is {obj_torch}.")

v_scipy is [1.00000000e-04 6.26068083e-01], obj_scipy is 1.2521713310658997.
v_torch is [1.98235756e-19 6.26089900e-01], obj_torch is 1.2521650483509288.


In [26]:
from classes import Coordinate, Region, Demands_generator, Polyhedron, append_df_to_csv
import time
import torch
import torchquad
from problem14 import minimize_problem14
from problem7 import minimize_problem7

def findWorstTSPDensity(region, demands, thetarange, t, epsilon, tol: float=1e-4):
    '''
    Algorithm by Carlsson, Behroozl, and Mihic, 2018.
    Code by Yidi Miao, 2023.

    This algorithm (find the worst TSP density) takes as input a compact planar region containing 
    a set of n distinct points, a distance threshold t, and a tolerance epsilon.

    Input: A compact, planar region Rg containing a set of distinct points x1, x2,..., xn, which are 
    interpreted as an empirical distribution f_hat, a distance parameter t, and a tolerance epsilon.

    Output: An epsilon-approximation of the distribution f* that maximizes iint_Rg sqrt(f(x)) dA 
    subject to the constraint that D(f_hat, f) <= t.

    This is a standard analytic center cutting plane method applied to problem (13), which has an 
    n-dimensional variable space.
    '''

    start, end = thetarange
    n = demands.shape[0]
    simpson = torchquad.Simpson()
    UB, LB = np.inf, -np.inf
    lambdas_bar = np.zeros(n)
    polyhedron = Polyhedron(np.eye(n), region.diam*np.ones(n), np.ones((1, n)), 0, n)
    k = 1
    while (abs(UB - LB) > epsilon and k < 10):
        print(f'Looking for worst-distribution on {[start, end]}:\nIteration {k} begins: \n')
        starttime = time.time()
        lambdas_bar, lambdas_bar_func_val = polyhedron.find_analytic_center(lambdas_bar)
        print(f'Find analytic center: Lambdas_bar is {lambdas_bar}, with value {lambdas_bar_func_val}, took {time.time() - starttime}s.\n')
        with open("./timerecords/find_analytic_center.txt", "a") as f:
            f.write(f"{time.time() - starttime}\n")

        demands_locations = np.array([demands[i].get_cdnt() for i in range(len(demands))])

        '''Build an upper bounding f_bar for the original problem (4).'''
        time_before_minimize_problem14 = time.time()
        v_bar, problem14_func_val = minimize_problem14(demands, thetarange, lambdas_bar, t, region.radius)
        with open("./timerecords/minimize_problem14.txt", "a") as f:
            f.write(f"{time.time() - time_before_minimize_problem14}\n")

        time_before_ub_integral = time.time()
        upper_integrand = lambda X: X[:, 0]*torch.sqrt(f_bar(X, demands_locations, lambdas_bar, v_bar))
        UB = simpson.integrate(upper_integrand, dim=2, N=999999, integration_domain=[[0, region.radius],[start, end]], backend='torch').item()
        with open("./timerecords/ub_integral.txt", "a") as f:
            f.write(f"{time.time() - time_before_ub_integral}\n")

        print(f'UB is {UB}, took {time.time() - starttime}s.\n')

        if UB < 0:
            print(f'UB is negative: {UB}.')
            print(f'v_bar is {v_bar}, problem14_func_val is {problem14_func_val}.')
            break

        '''Build an lower bounding f_tilde that us feasible for (4) by construction.'''
        time_before_minimize_problem7 = time.time()
        v_tilde, problem7_func_val = minimize_problem7(lambdas_bar, demands, thetarange, t, region.radius, tol)
        with open("./timerecords/minimize_problem7.txt", "a") as f:
            f.write(f"{time.time() - time_before_minimize_problem7}\n")

        time_before_lb_integral = time.time()
        lower_integrand = lambda X: X[:, 0]*torch.sqrt(f_tilde(X, demands_locations, lambdas_bar, v_tilde))
        LB =simpson.integrate(lower_integrand, dim=2, N=999999, integration_domain=[[0, region.radius],[start, end]], backend='torch').item()
        with open("./timerecords/lb_integral.txt", "a") as f:
            f.write(f"{time.time() - time_before_lb_integral}\n")
        
        print(f'LB is {LB}, took {time.time() - starttime}s.\n')

        '''Update g.'''
        g = np.zeros(len(demands))
        time_before_g_integral = time.time()
        for i in range(len(demands)):
            integrandi = lambda X: X[:, 0]*f_bar(X, demands_locations, lambdas_bar, v_bar, masked=True, j=i+1) 
            g[i] = simpson.integrate(integrandi, dim=2, N=999999, integration_domain=[[0, region.radius], [start, end]], backend='torch').item()
        with open("./timerecords/g_integral.txt", "a") as f:
            f.write(f"{time.time() - time_before_g_integral}\n")
        
        '''Update polyheron Lambda to get next analytic center.'''
        polyhedron.add_ineq_constraint(g, g.T @ lambdas_bar)

        print(f'End of iteration {k}.\n  The whole iteration took {time.time() - starttime}s.\n')
        k += 1

    return lambda X: f_tilde(X, demands_locations, lambdas_bar, v_tilde)


@nb.jit(nopython=True)
def norm_func(x, y):
    return np.sqrt(np.sum(np.square(x - y)))

def f_bar(X, demands_locations, lambdas_bar, v_bar, masked=False, j=-1):
    '''
    X is a n-by-2 matrix, the first column is r, the second column is theta.
    '''
    dtype = X.dtype
    lambdas = torch.tensor(lambdas_bar, dtype=dtype)
    demands_locations = torch.tensor(demands_locations, dtype=dtype)
    x_cdnt = X.clone()
    x_cdnt[:, 0] = X[:, 0]*torch.cos(x_cdnt[:, 1])
    x_cdnt[:, 1] = X[:, 0]*torch.sin(x_cdnt[:, 1])
    norms = torch.cdist(x_cdnt, demands_locations, p=2)
    modified_norms, modified_norms_indices = torch.min(norms - lambdas, dim=1)
    raw_intgrd = 1 / (4*torch.square(v_bar[0]*modified_norms + v_bar[1]))
    if masked:        
        mask = modified_norms_indices != j-1
        masked_intgrd = raw_intgrd
        masked_intgrd[mask] = 0
        return masked_intgrd
    return raw_intgrd


def f_tilde(X, demands_locations, lambdas_bar, v_tilde):
    '''
    X is a n-by-2 matrix, the first column is r, the second column is theta.
    '''
    dtype = X.dtype
    lambdas, v1, v0 = torch.tensor(lambdas_bar, dtype=dtype), torch.tensor(v_tilde[1:], dtype=dtype), v_tilde[0]
    demands_locations = torch.tensor(demands_locations, dtype=dtype)
    x_cdnt = X.clone()
    x_cdnt[:, 0] = X[:, 0]*torch.cos(x_cdnt[:, 1])
    x_cdnt[:, 1] = X[:, 0]*torch.sin(x_cdnt[:, 1])
    norms = torch.cdist(x_cdnt, demands_locations, p=2)
    modified_norms, modified_norms_indices = torch.min(norms - lambdas, dim=1)
    corresponding_norms = norms[torch.arange(norms.shape[0]), modified_norms_indices]
    raw_intgrd = 1 / (4*torch.square(v0*corresponding_norms + v1[modified_norms_indices]))
    return raw_intgrd


region = Region(1)
depot = Coordinate(2, 0.3)
t, epsilon = 0.25, 0.1

thetarange = (3.147444264116321 - 2*np.pi, 0.00012460881117814946)
demands = np.array([Demand(Coordinate(r = 0.1802696888767692, theta = 4.768809396779301), 1),
           Demand(Coordinate(r = 0.019475241487624584, theta = 5.1413757057591045), 1),
           Demand(Coordinate(r = 0.012780814590608647, theta = 4.478189127165961), 1),
           Demand(Coordinate(r = 0.4873716073198716, theta = 3.7670422583826495), 1),
           Demand(Coordinate(r = 0.10873607186897494, theta = 5.328009178184025), 1),
           Demand(Coordinate(r = 0.8939041702850812, theta = 4.5103794163992506), 1),
           Demand(Coordinate(r = 0.8571542470728296, theta = 3.7828800007876318), 1),
           Demand(Coordinate(r = 0.16508661759522714, theta = 3.4707299112070515), 1),
           Demand(Coordinate(r = 0.6323340138234961, theta = 5.963386240530588), 1),
           Demand(Coordinate(r = 0.020483612791232675, theta = 6.19945137229428), 1),
           Demand(Coordinate(r = 0.15791230667493694, theta = 5.004153427569559), 1)])
demands_locations = np.array([demands[i].get_cdnt() for i in range(len(demands))])
lambdas = np.zeros(demands.shape[0])

optimal_ftilde = findWorstTSPDensity(region, demands, thetarange, t, epsilon, tol=1e-4)


Looking for worst-distribution on [-3.1357410430632653, 0.00012460881117814946]:
Iteration 1 begins: 

Find analytic center: Lambdas_bar is [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], with value -7.624624486158025, took 0.0006852149963378906s.

UB is 1.252150297164917, took 0.4859468936920166s.

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0460491390275553
            Iterations: 18
            Function evaluations: 31
            Gradient evaluations: 18
LB is 1.048218846321106, took 75.54212498664856s.

End of iteration 1.
  The whole iteration took 75.90299606323242s.

Looking for worst-distribution on [-3.1357410430632653, 0.00012460881117814946]:
Iteration 2 begins: 

Find analytic center: Lambdas_bar is [ 0.1671064   0.51918531  0.51360964 -0.1566549   0.42367315 -0.72386192
 -0.22901394  0.40783681 -1.85184451  0.52064143  0.40932253], with value -6.710047167722275, took 0.006017923355102539s.



/Users/miaoyidi/Desktop/HCT/Partition/last-mile-optimal-n-parition/classes.py:133: RuntimeWarning: invalid value encountered in log
  objective = lambda x: -np.sum(np.log(self.b - self.A @ x + 1e-6))  # To ensure log(b - A @ x) is defined.
/Users/miaoyidi/miniconda3/envs/HCT/lib/python3.11/site-packages/torchquad/integration/utils.py:255: UserWarning: DEPRECATION WARNING: In future versions of torchquad, an array-like object will be returned.
  warnings.warn(


UB is 1.252150297164917, took 0.5027637481689453s.



/Users/miaoyidi/miniconda3/envs/HCT/lib/python3.11/site-packages/torchquad/integration/simpson.py:77: UserWarning: N per dimension cannot be even due to necessary subdivisions. N per dim will now be changed to the next lower integer, i.e. 3162 -> 3161.
  warnings.warn(


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.8191587152793419
            Iterations: 15
            Function evaluations: 23
            Gradient evaluations: 15
LB is 0.8112115263938904, took 61.340813875198364s.

End of iteration 2.
  The whole iteration took 61.67294692993164s.

Looking for worst-distribution on [-3.1357410430632653, 0.00012460881117814946]:
Iteration 3 begins: 

Find analytic center: Lambdas_bar is [ 0.54905877  0.59784217  0.84501601  0.24348338  0.77727213 -0.32955369
  0.05583497 -0.32603295 -1.66799151  0.2332242  -0.97815348], with value -5.862344565790702, took 0.005124092102050781s.



...

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6127916722993346
            Iterations: 15
            Function evaluations: 18
            Gradient evaluations: 15
LB is 0.6123137474060059, took 61.895729064941406s.

End of iteration 8.
  The whole iteration took 62.241146087646484s.

Looking for worst-distribution on [-3.1357410430632653, 0.00012460881117814946]:
Iteration 9 begins: 

Find analytic center: Lambdas_bar is [-0.07018621 -0.84636041  0.3147807   0.50515422  0.17927765  0.1451705
  0.15073839  0.37222702 -1.2576766   0.88107545 -0.37420071], with value 37.413913277003566, took 0.0013599395751953125s.



/Users/miaoyidi/miniconda3/envs/HCT/lib/python3.11/site-packages/torchquad/integration/utils.py:255: UserWarning: DEPRECATION WARNING: In future versions of torchquad, an array-like object will be returned.
  warnings.warn(


UB is 1.252150297164917, took 0.4656360149383545s.



/Users/miaoyidi/miniconda3/envs/HCT/lib/python3.11/site-packages/torchquad/integration/simpson.py:77: UserWarning: N per dimension cannot be even due to necessary subdivisions. N per dim will now be changed to the next lower integer, i.e. 3162 -> 3161.
  warnings.warn(


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6127916722993346
            Iterations: 15
            Function evaluations: 18
            Gradient evaluations: 15
LB is 0.6123137474060059, took 62.901196002960205s.

End of iteration 9.
  The whole iteration took 63.23278999328613s.



In [27]:
integrand = lambda X: X[:, 0]*torch.sqrt(optimal_ftilde(X))
simpson = torchquad.Simpson()
area = simpson.integrate(integrand, dim=2, N=999999, integration_domain=[[0, region.radius],[thetarange[0], thetarange[1]]], backend='torch').item()
print(f"area is {area}.")

area is 0.6123137474060059.


/Users/miaoyidi/miniconda3/envs/HCT/lib/python3.11/site-packages/torchquad/integration/utils.py:255: UserWarning: DEPRECATION WARNING: In future versions of torchquad, an array-like object will be returned.
  warnings.warn(
